# p08. Join, MErge/Combine, Transform

## 1. 계층적 index

### : 계층적인 정보를 index로 사용하고 싶을때

In [3]:
import numpy as np
import pandas as pd

### (1) Series

In [8]:
data = pd.Series(np.random.randn(10),
                index=[['a','a','a','a','b','b','c','c','d','d'],
                [1,2,3,1,2,3,1,2,2,3]])
data

a  1   -0.260711
   2   -0.347364
   3    0.555264
   1    0.848994
b  2   -0.669375
   3   -0.581415
c  1   -0.288256
   2   -0.594277
d  2   -0.297283
   3    0.313274
dtype: float64

In [10]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 1, 2]])

In [12]:
data['b']

2   -0.669375
3   -0.581415
dtype: float64

In [18]:
data[1]

-0.3473643494459686

In [19]:
data['b':'c'] #가장 바깥층의 index 명의 범위 인덱싱

b  2   -0.669375
   3   -0.581415
c  1   -0.288256
   2   -0.594277
dtype: float64

In [21]:
data[('b',3)]

-0.5814149397778561

In [23]:
data['b',3]

-0.5814149397778561

In [25]:
data[:,2]

a   -0.347364
b   -0.669375
c   -0.594277
d   -0.297283
dtype: float64

In [29]:
#(2) DataFrame에서 계층적 인덱싱 : idnex, coumn 모두 2차원

df = pd.DataFrame(np.arange(12).reshape(4,3),
                 index=[['a','a','b','b'],[1,2,1,2]],
                 columns=[['Seoul','Seoul',"Busan"],["Green","Red","Green"]])
df


Seoul     Busan
    Green Red Green
a 1     0   1     2
  2     3   4     5
b 1     6   7     8
  2     9  10    11

In [32]:
# 복수 계층의 인덱스와 컬럼에 이름을 붙일 때 (리스트 형식)
df.index.names=['key1','key2']
df.columns.names=['city','color']
df

city      Seoul     Busan
color     Green Red Green
key1 key2                
a    1        0   1     2
     2        3   4     5
b    1        6   7     8
     2        9  10    11

In [34]:
#Data Frame 열에 대한 계층적인 인덱싱
df['Seoul']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [36]:
df.loc['a']

city  Seoul     Busan
color Green Red Green
key2                 
1         0   1     2
2         3   4     5

In [38]:
df.loc['a',1]

city   color
Seoul  Green    0
       Red      1
Busan  Green    2
Name: (a, 1), dtype: int32

In [41]:
df.sort_index(axis=0,level='key2')

city      Seoul     Busan
color     Green Red Green
key1 key2                
a    1        0   1     2
b    1        6   7     8
a    2        3   4     5
b    2        9  10    11

In [43]:
df.sort_index(axis=1,level='color')

city      Busan Seoul    
color     Green Green Red
key1 key2                
a    1        2     0   1
     2        5     3   4
b    1        8     6   7
     2       11     9  10

In [45]:
#Busan 계층의 Green 열의 성분들을 오름차순
df.sort_values(by=("Busan","Green"))

city      Seoul     Busan
color     Green Red Green
key1 key2                
a    1        0   1     2
     2        3   4     5
b    1        6   7     8
     2        9  10    11

### DataFrame의 특정열의 성분을 계층적 인덱스로 변환

In [49]:
df2=pd.DataFrame({'a':range(7),'b':range(7,0,-1),'c':['one','one','one','two','two','two','two'],
                 'd':[0,1,2,0,1,2,3]})
df2

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [52]:
df3=df2.set_index(['c','d'])
df3

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [54]:
df2.set_index(['c','d'],drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

## 2. 변형 : 재변형& 피벗(pivot) 334
### dataframe 모양 변형

In [56]:
df4=pd.DataFrame(np.arange(6).reshape(2,3),
                index=['Seoul','Busan'],
                columns=['one','two','three'])
df4

,one,two,three
Seoul,0,1,2
Busan,3,4,5


In [58]:
#stack(): 최하위 컬럼(number)=> 최하위 인덱스(city)
# 단일컬럼 -> 단일인덱스뺴

df4.stack()

Seoul  one      0
       two      1
       three    2
Busan  one      3
       two      4
       three    5
dtype: int32

In [60]:
#unstack(): index 취하위 -> 최하위 컬럼
df4.unstack()

one    Seoul    0
       Busan    3
two    Seoul    1
       Busan    4
three  Seoul    2
       Busan    5
dtype: int32

## 3. 데이터 합치기 (merge,concatenate)

### (1) merge

In [62]:
df1=pd.DataFrame({'key':list("bbaccab"),"data1":range(7)})

In [64]:
df2=pd.DataFrame({"key":list("abc"),'data2':range(3)})

In [67]:
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,5,0
5,c,3,2
6,c,4,2


In [69]:
pd.merge(df1,df2,on='key',how='left')

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,c,3,2
4,c,4,2
5,a,5,0
6,b,6,1


In [71]:
#how='left' left.join 같은 개념 - 왼쪽에 있는 df1이 기준이 되어서 고정,
# df2 동일한 키값을 가질떄마다 여러번 달라붙는다

pd.merge(df1,df2,on='key',how='outer')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,5,0
5,c,3,2
6,c,4,2


In [73]:
pd.merge(df1,df2,left_on='lkey',right_on='rkey')

KeyError: 'rkey'